# Risk Analysis

Comprehensive risk assessment through Monte Carlo simulation, drawdown analysis, rolling metrics, and stress testing.

In [15]:
from datetime import timedelta
import pandas as pd
from clyptq import CostModel, Constraints
from clyptq.analytics.risk.monte_carlo import MonteCarloSimulator
from clyptq.analytics.performance.drawdown import DrawdownAnalyzer
from clyptq.analytics.performance.rolling import RollingMetricsCalculator
from clyptq.data.loaders.ccxt import load_crypto_data
from clyptq.trading.engine import BacktestEngine
from clyptq.trading.execution import BacktestExecutor
from clyptq.trading.factors.library.momentum import MomentumFactor
from clyptq.trading.factors.library.volatility import VolatilityFactor
from clyptq.trading.portfolio.constructors import TopNConstructor
from clyptq.trading.strategy.base import SimpleStrategy

## 1. Run Backtest

In [16]:
symbols = [
    "BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT",
    "ADA/USDT", "AVAX/USDT", "DOGE/USDT", "DOT/USDT", "MATIC/USDT",
]

store = load_crypto_data(symbols=symbols, exchange="binance", timeframe="1d", days=720)
date_range = store.get_date_range()
start = date_range.end - timedelta(days=365)
end = date_range.end

class MomentumStrategy(SimpleStrategy):
    def __init__(self):
        super().__init__(
            factors_list=[MomentumFactor(lookback=30), VolatilityFactor(lookback=30)],
            constructor=TopNConstructor(top_n=3),
            constraints_obj=Constraints(max_position_size=0.40, max_gross_exposure=1.0, min_position_size=0.10, max_num_positions=5),
            schedule_str="weekly",
            warmup=35,
            name="Momentum",
        )

cost_model = CostModel(maker_fee=0.001, taker_fee=0.001, slippage_bps=5.0)
executor = BacktestExecutor(cost_model)
engine = BacktestEngine(MomentumStrategy(), store, executor, 100000.0)

print("Running backtest...")
result = engine.run(start=start, end=end, verbose=False)

m = result.metrics
print(f"\nBacktest: {m.total_return:.2%} return, {m.sharpe_ratio:.3f} Sharpe")

Running backtest...
Fill rejected: Insufficient cash for ETH/USDT: need 32072.83, have 32040.89
Fill rejected: Insufficient cash for XRP/USDT: need 24231.15, have 24207.02

Backtest: -27.27% return, -1.330 Sharpe


## 2. Monte Carlo Simulation

Bootstrap 1000 simulations to estimate distribution of outcomes.

In [17]:
mc = MonteCarloSimulator(num_simulations=1000, random_seed=42)
mc_result = mc.run(result)

print("MONTE CARLO SIMULATION (1000 runs)")
print("=" * 80)

print(f"\nReturn Distribution:")
print(f"  Mean:               {mc_result.mean_return:>10.2%}")
print(f"  Median:             {mc_result.median_return:>10.2%}")
print(f"  Std Dev:            {mc_result.std_return:>10.2%}")
print(f"  5th percentile:     {mc_result.ci_5_return:>10.2%}")
print(f"  95th percentile:    {mc_result.ci_95_return:>10.2%}")

print(f"\nRisk Metrics:")
print(f"  P(Loss):            {mc_result.probability_of_loss:>10.2%}")
print(f"  CVaR (5%):          {mc_result.expected_shortfall_5:>10.2%}")
print(f"  VaR (5%):           {mc_result.ci_5_return:>10.2%}")

print(f"\nDrawdown Distribution:")
print(f"  5th percentile:     {mc_result.max_drawdown_5:>10.2%}")
print(f"  Median:             {mc_result.max_drawdown_50:>10.2%}")
print(f"  95th percentile:    {mc_result.max_drawdown_95:>10.2%}")

print(f"\nSharpe Distribution:")
print(f"  Mean:               {mc_result.mean_sharpe:>10.3f}")
print(f"  90% CI:             [{mc_result.ci_5_sharpe:.3f}, {mc_result.ci_95_sharpe:.3f}]")

MONTE CARLO SIMULATION (1000 runs)

Return Distribution:
  Mean:                  -24.05%
  Median:                -26.40%
  Std Dev:                20.61%
  5th percentile:        -52.76%
  95th percentile:        15.10%

Risk Metrics:
  P(Loss):                88.10%
  CVaR (5%):             -57.79%
  VaR (5%):              -52.76%

Drawdown Distribution:
  5th percentile:         14.21%
  Median:                 33.58%
  95th percentile:        54.30%

Sharpe Distribution:
  Mean:                   -4.272
  90% CI:             [-11.158, 2.767]


## 3. Drawdown Analysis

Understand when and how much we lose during drawdown periods.

In [18]:
dd_analyzer = DrawdownAnalyzer(min_drawdown=0.05)
dd_analysis = dd_analyzer.analyze(result)

print("\nDRAWDOWN ANALYSIS")
print("=" * 80)
print(f"Max Drawdown: {dd_analysis.max_drawdown:.2%}")
print(f"Avg Drawdown: {dd_analysis.avg_drawdown:.2%}")
print(f"Number of periods: {len(dd_analysis.drawdown_periods)}")

if dd_analysis.drawdown_periods:
    print("\nTop 5 Drawdown Periods:")
    print(f"{'Start':<12} {'End':<12} {'Recovery':<12} {'Depth':<10} {'Duration':<10} {'Recovery Days':<15}")
    print("-" * 80)
    
    for period in dd_analysis.drawdown_periods[:5]:
        recovery_str = period.recovery.date() if period.recovery else "Ongoing"
        recovery_days = period.recovery_days if period.recovery_days else "-"
        print(
            f"{period.start.date()!s:<12} "
            f"{period.end.date()!s:<12} "
            f"{recovery_str!s:<12} "
            f"{period.depth:>8.2%}  "
            f"{period.duration_days:>8}  "
            f"{recovery_days!s:>13}"
        )


DRAWDOWN ANALYSIS
Max Drawdown: 38.89%
Avg Drawdown: 38.89%
Number of periods: 1

Top 5 Drawdown Periods:
Start        End          Recovery     Depth      Duration   Recovery Days  
--------------------------------------------------------------------------------
2025-10-13   2025-12-29   Ongoing       -38.89%        12              -


## 4. Rolling Risk Metrics

Track how risk metrics change over time with rolling windows.

In [19]:
rolling_calc = RollingMetricsCalculator(window=4)  # 4 weeks for weekly strategy
rolling = rolling_calc.calculate(result)

print("\nROLLING METRICS (4-week window)")
print("=" * 80)

sharpe_series = pd.Series(rolling.sharpe_ratio)
sortino_series = pd.Series(rolling.sortino_ratio)
vol_series = pd.Series(rolling.volatility)

print(f"\nSharpe Ratio:")
print(f"  Mean:               {sharpe_series.mean():>10.3f}")
print(f"  Std:                {sharpe_series.std():>10.3f}")
print(f"  Min:                {sharpe_series.min():>10.3f}")
print(f"  Max:                {sharpe_series.max():>10.3f}")

print(f"\nSortino Ratio:")
print(f"  Mean:               {sortino_series.mean():>10.3f}")
print(f"  Std:                {sortino_series.std():>10.3f}")

print(f"\nVolatility:")
print(f"  Mean:               {vol_series.mean():>10.2%}")
print(f"  Range:              [{vol_series.min():.2%}, {vol_series.max():.2%}]")

if rolling.max_drawdown:
    dd_series = pd.Series(rolling.max_drawdown)
    print(f"\nRolling Drawdown:")
    print(f"  Mean:               {dd_series.mean():>10.2%}")
    print(f"  Max:                {dd_series.min():>10.2%}")


ROLLING METRICS (4-week window)

Sharpe Ratio:
  Mean:                   -5.659
  Std:                    10.344
  Min:                   -19.967
  Max:                    16.342

Sortino Ratio:
  Mean:                  -15.633
  Std:                    23.677

Volatility:
  Mean:                  135.70%
  Range:              [86.76%, 183.45%]

Rolling Drawdown:
  Mean:                  -11.20%
  Max:                   -23.71%


/Users/jaden/Desktop/clypt-llm/clypt-trading-engine/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:219: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/jaden/Desktop/clypt-llm/clypt-trading-engine/venv/lib/python3.13/site-packages/numpy/_core/_methods.py:211: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## 5. Risk Limits

Production thresholds for live trading.

In [ ]:
print("\nRISK LIMIT VALIDATION")
print("=" * 80)

# Define limits
limits = {
    "Max Drawdown": (m.max_drawdown, 0.30),  # (actual, limit)
    "Min Sharpe": (m.sharpe_ratio, 0.5),
    "Max Volatility": (m.volatility, 0.50),
    "Min Win Rate": (m.win_rate, 0.40),
    "P(Loss)": (mc_result.probability_of_loss, 0.25),
    "CVaR (5%)": (abs(mc_result.expected_shortfall_5), 0.15),
}

all_pass = True
for name, (actual, limit) in limits.items():
    if "Min" in name:
        passed = actual >= limit
        status = "PASS" if passed else "FAIL"
    else:
        passed = actual <= limit
        status = "PASS" if passed else "FAIL"
    
    all_pass = all_pass and passed
    
    if isinstance(actual, float) and actual < 10:
        if "Rate" in name or "%" in name or "P(" in name:
            print(f"{name:<20} {actual:>8.2%} / {limit:>8.2%}  {status}")
        else:
            print(f"{name:<20} {actual:>8.3f} / {limit:>8.3f}  {status}")
    else:
        print(f"{name:<20} {actual:>8.2%} / {limit:>8.2%}  {status}")

print("=" * 80)
if all_pass:
    print("\nAll risk limits passed - APPROVED for production")
else:
    print("\nSome limits failed - DO NOT deploy to production")

## 6. Summary

Monte Carlo simulates 1000 possible outcomes to assess tail risk and capital allocation. Drawdown analysis tracks recovery patterns and maximum loss periods. Rolling metrics monitor stability over time and detect regime changes. Risk limits define production deployment thresholds.

## Next Steps

- **08_paper_trading.ipynb**: Deploy to paper trading
- Set up real-time risk monitoring
- Define kill switches (max DD, max loss per day)
- Regular risk reviews (weekly/monthly)